In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle

In [24]:
regularization_strength = 100
learning_rate = 0.000001

In [2]:
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularization_strength * (np.sum(distances))

    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost


In [3]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (regularization_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw


In [37]:
def sgd(features, outputs):
    max_epochs = 500000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.001  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights



In [38]:

    print("reading dataset...")
    # read data in pandas (pd) data frame
    data = pd.read_csv('diagnosis.csv')
    data = data[0:20]
    # drop last column (extra column added by pd)
    # and unnecessary first column (id)
    data.drop(data.columns[[-1, 0]], axis=1, inplace=True)

    print("applying feature engineering...")
    # convert categorical labels to numbers
    diag_map = {'M': 1.0, 'B': -1.0}
    data['diagnosis'] = data['diagnosis'].map(diag_map)

    # put features & outputs in different data frames
    Y = data.loc[:, 'diagnosis']
    X = data.iloc[:, 1:]

    # filter features
    #remove_correlated_features(X)
    #remove_less_significant_features(X, Y)

    # normalize data for better convergence and to prevent overflow
    X_normalized = StandardScaler().fit_transform(X.values)
    X = pd.DataFrame(X_normalized)

    # insert 1 in every row for intercept b
    X.insert(loc=len(X.columns), column='intercept', value=1)

    # split data into train and test set
    print("splitting dataset into train and test sets...")
    #tts(X, Y, test_size=0.2, random_state=42)

    # train the model
    print("training started...")
    W = sgd(X.to_numpy(), Y.to_numpy())
    print("training finished.")
    print("weights are: {}".format(W))
    

reading dataset...
applying feature engineering...
splitting dataset into train and test sets...
training started...
Epoch is: 1 and Cost is: 156072.1943261366
Epoch is: 2 and Cost is: 120917.37594717379
Epoch is: 4 and Cost is: 67725.93541933036
Epoch is: 8 and Cost is: 25390.426338853835
Epoch is: 16 and Cost is: 15464.83773143997
Epoch is: 32 and Cost is: 11344.005877032041
Epoch is: 64 and Cost is: 842.5344708606142
Epoch is: 128 and Cost is: 2.128936606596986
Epoch is: 256 and Cost is: 2.118064302584236
Epoch is: 512 and Cost is: 2.096485982887454
Epoch is: 1024 and Cost is: 2.21265877981558
Epoch is: 2048 and Cost is: 2.1550717057978095
Epoch is: 4096 and Cost is: 2.1515976810241204
Epoch is: 8192 and Cost is: 2.106278279107557
Epoch is: 16384 and Cost is: 1.9441113690433887
Epoch is: 32768 and Cost is: 2.039532901222637
Epoch is: 65536 and Cost is: 1.9714393498676208
Epoch is: 131072 and Cost is: 2.045050489449382
Epoch is: 262144 and Cost is: 2.0667543443252674
Epoch is: 499999

In [35]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l1", max_iter=4000000,alpha=10000,verbose=True)
clf.fit(X, Y)
clf.coef_

##not sure why this doesn't work to verify??? will solve ourselves and compare 
#to the SGD implementation above, sklearn doesn't seem to beat it anyways

-- Epoch 1
Norm: 0.60, NNZs: 0, Bias: 0.100315, T: 20, Avg. loss: 0.914807
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.60, NNZs: 0, Bias: 0.100378, T: 40, Avg. loss: 0.909687
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 0.60, NNZs: 0, Bias: 0.100415, T: 60, Avg. loss: 0.909643
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 0.60, NNZs: 0, Bias: 0.100441, T: 80, Avg. loss: 0.909615
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 0.60, NNZs: 0, Bias: 0.100462, T: 100, Avg. loss: 0.909594
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 0.60, NNZs: 0, Bias: 0.100478, T: 120, Avg. loss: 0.909577
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 0.60, NNZs: 0, Bias: 0.100492, T: 140, Avg. loss: 0.909564
Total training time: 0.00 seconds.
Convergence after 7 epochs took 0.00 seconds


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [33]:
print(compute_cost(clf.coef_[0],X,Y))

200000.0


In [44]:
print(X.shape)

(20, 31)


In [91]:
import math

def dual_coord_l1(X, y, max_steps, C=.25):
    u=C
    n_features = X.shape[1]
    print(n_features)
    n_samples = X.shape[0]
    print(n_samples)
    X=X.transpose()
    w = np.zeros(n_features)
    alpha = np.zeros(n_samples)
    for o in range(max_steps) : #outer iteration
        for i in range(n_samples): #inner iteration
            #step a
            G = y[i]*w.transpose().dot(X[i])-1#+alpha[i] * 0 #G = yiwT xi − 1 + Diiαi
            #step b of algorithm 1
            PG = 0
            if (alpha[i]==0):
                PG = min(G,0)
            if (alpha[i]==u):
                PG = max(G,0)
            if (alpha[i] < u and alpha[i] == 0):
                PF=G
            #step c
            if PG != 0: 
                ahat = alpha[i]
                #for L1, Qhatii = Qii+Dii = Qii+0
                qhatii = ((y[i]**2)*X[i].dot(X[i]))
                alpha[i] = min(max(alpha[i]-(G/qhatii),0),u)
                w = w + ((alpha[i]-ahat)*y[i])*X[i]
        if o % 100 == 0:
            print(compute_cost(w,X.transpose(),Y))
    return w

In [92]:
dual_coord_l1(X,Y,500,C=100)

31
20
70310.59929354131
70310.59929354131
70310.59929354131
70310.59929354131
70310.59929354131


0            0.044571
1            0.112143
2            0.054524
3            0.032979
4            0.091592
5            0.103191
6           -0.006495
7            0.048300
8            0.055544
9            0.105741
10           0.075186
11           0.086485
12           0.054400
13           0.028284
14          -0.113617
15           0.009392
16          -0.061251
17          -0.015820
18          -0.074891
19           0.039678
20           0.113572
21           0.152541
22           0.096058
23           0.086144
24           0.094664
25           0.039255
26          -0.011528
27           0.079710
28           0.000638
29           0.080198
intercept    0.714651
dtype: float64